# Wilcoxon rank-sum test
by Tonatiuh Rangel   
In this notebook, I show a numerical example and give a brief explanation of the Wilcoxon rank-sum test.   

## Contents
1. [Wilcoxon rank-sum test](#WRS)    
    1.1 [Ranking the data](#ranking)    
    1.2 [Rank sums](#rank-sums)   
    1.3 [Test statistic](#test-statistic)     
    1.4 [z-score](#z-score)
    1.5 [Scipy example](#scipy)   
2. [Mann-Whitney test](#MW)   
    2.1 [Scipy example](#MW_scipy)   



<a id='WRS'></a>
## Wilcoxon rank-sum test
As for other non-parametric tests, the Wilcoxon rank-sum test analyses the ranks of data instead of the bare data.    
This is equivalent to the [Mann-Whitney test](#MW).  


### Assumptions    
This is a non-parametric test, i.e., an assumption-free test.   


### Hypothesis
|Set|	Null hypothesis	| Alternative hypothesis |No. of tails |
|---|:-----------------:|:----------------------:|:-----------:|
|1	| $\mu_1 - \mu_2 = 0$ | $\mu_1 - \mu_2 \neq 0$ |	2|

This is a two-tailed test.    

### Test method

* Mean of rank-sum   
$\overline{W}_s = n_1(n_1+n_2+1)/2$


* Standard error    
SE$_\overline{W_s}= \sqrt{n_1 n_2 (n_1 + n_2 + 1)/12}$


* $z$-score    
$z = \frac{\displaystyle X - \overline{X}}{\displaystyle s} = 
\frac{\displaystyle W_s - \overline{W_s}}
{\displaystyle \textrm{SE}_{\overline{W_s}}}$



### Usage   
* This is used to compare the means of two distributions.    
* The non-parametric equivalent of the independent t-test, please see my previous notebook [Difference between means](https://github.com/trangel/stats-with-python/blob/master//Difference%20between%20means.ipynb).   

### Further reading    
* Chapter 15 of Discovering Statistics Using SPSS by Andy Field.   

### Numerical example    
To start with this notebook, let's use data from table 15.1 of Discovering Statistics Using SPSS by Andy Field.    
This data is a drug experiment with 2 groups of 10 participants.
A group is giving a drug (Ecstasy) for depresion and the other are only allowed to drink alcohol, the levels of depression are then meassured using the Beck Depression Inventory the day after and midweek. 



In [1]:
import pandas as pd
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# Read dataset locally
#df = pd.read_csv("../data/data-for-drug-experiment.csv",index_col=0)
    
# Read dataset from url:
import io
import requests
url="https://raw.githubusercontent.com/trangel/stats-with-python/master/data/data-for-drug-experiment.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),index_col=0)
    
df

,Drug,BDI Sunday,BDI Wednesday
Participant,,,
1,Ecstasy,15,28
2,Ecstasy,35,35
3,Ecstasy,16,35
4,Ecstasy,18,24
5,Ecstasy,19,39
6,Ecstasy,17,32
7,Ecstasy,27,27
8,Ecstasy,16,29
9,Ecstasy,13,36


<a id='ranking'></a>
## Ranking the data

First we need to compute the ranks for each of the distributions.   
Ranking the data is done as follows: finding the lowest score and giving it a rank of 1, the next lowest score is given a rank of 2, etc...   
In case of tied-ranks (repeated scores), the rank is averaged.    

To rank the data we start by sorting the data.    
Then we treat tied-ranks    

In [2]:
def rank_distribution(dist,column):
    """Get distribution ranks for non-parametric test
    Column: str, column name to get ranking
    Input: dataframe with distribution values.
    Output: dataframe including ranks
    """
    # Sort values
    dist.sort_values(by=column,inplace=True)
    ## Convert series to dataframe, to add ranks
    #dist = dist.to_frame()
    dist['Potential rank']=0
    jj=0
    for ii in dist.index:
        jj = jj + 1
        dist.loc[ii,'Potential rank']=jj
        
    # Get actual ranks by averaging ranks of repeated scores  
    # rank table with average (mean) values of ranks group by BDI scores:   
    rank = dist.groupby(column).agg({'Potential rank':'mean'})
    rank.columns=['Rank']
    #
    dist = dist.join(rank,on=column)

    return dist

In [3]:
BDI_Sunday = df[['Drug','BDI Sunday']].copy()
BDI_Sunday = rank_distribution(BDI_Sunday,'BDI Sunday')

BDI_Sunday.head(6)

,Drug,BDI Sunday,Potential rank,Rank
Participant,,,,
16,Alcohol,13,1,1.5
9,Ecstasy,13,2,1.5
17,Alcohol,14,3,3.0
1,Ecstasy,15,4,5.0
14,Alcohol,15,5,5.0
12,Alcohol,15,6,5.0


As you can see, the first two rows have the same BDI and have a rank of 1.5 (the average of 1 and 2) and similar for rows 4 to 6 the rank is 5.0, the average of 4, 5 and 6.

We get ranks for the Wednesday's samples:

In [4]:
BDI_Wednesday = df[['Drug','BDI Wednesday']].copy()

BDI_Wednesday = rank_distribution(BDI_Wednesday,'BDI Wednesday')

BDI_Wednesday.head(6)

,Drug,BDI Wednesday,Potential rank,Rank
Participant,,,,
19,Alcohol,3,1,1.0
11,Alcohol,5,2,2.0
17,Alcohol,6,3,3.5
12,Alcohol,6,4,3.5
16,Alcohol,7,5,5.0
14,Alcohol,8,6,6.0


<a id='rank-sums'></a>
## Rank - sums  
Let's compute the rank sums $W_s$ 

In [5]:
# For Sunday's BDI:

rank_sum = BDI_Sunday[['Drug','Rank']].groupby('Drug').agg('sum')
rank_sum.columns=['Rank sum']
rank_sum_sunday = rank_sum
rank_sum


,Rank sum
Drug,
Alcohol,90.5
Ecstasy,119.5


<a id ='test-statistic'></a>
### Test-statistic
When the groups have unequal numbers of participants in them then $W_s$ is simply the sum of ranks in the group that contains the fewer people. When the group sizes are equal it's the value of the smaller summed rank.    
In this case, the group sizes are equal, and hence the lowest of the rank sum (90.5) is our test statistic $W_s$. 



In [6]:
W = rank_sum.values.min()
printmd('$W_s$ {}'.format(W))

$W_s$ 90.5

<a id='z-score'></a>
### $z$-score    
Remember    
$z = \frac{\displaystyle X - \overline{X}}{\displaystyle s} = 
\frac{\displaystyle W_s - \overline{W_s}}
{\displaystyle \textrm{SE}_{\overline{W_s}}}$

In [7]:
import numpy as np
n1 = 10
n2 = 10
W_mean = n1 *(n1 + n2 + 1.0)/2.0
SE  = (n1*n2 * (n1+n2+1.0)/12.0)**0.5

# Using the z-score equation the Sunday score is:   
z = (W - W_mean)/SE

printmd('** Sunday **')
printmd('$W_s$ {}'.format(W))
printmd('$W_s$ mean {}'.format(W_mean))
printmd('SE {}'.format(round(SE,2)))
printmd('$z$-score {}'.format(round(z,2)))


** Sunday **

$W_s$ 90.5

$W_s$ mean 105.0

SE 13.23

$z$-score -1.1

We need to look for the probability for this $z$-score on a table, or using the CDF for a t-distribution as below.   
Remember that we get the z-distribution using the t-distribution with large degrees of freedom.   

In [8]:
from scipy import stats
DF=10000  # set a large degrees of freedom.
pvalue = 2* stats.t.cdf(z,DF) # get the P value from the CDF of a t-distribution
# Multiply by two, since this is two-tailed test

printmd('p-value {}'.format(round(pvalue,4)))

p-value 0.2731

In [9]:
# Compute now the Wednesday z-score
# Using the z-score equation the Sunday score is:   

rank_sum = BDI_Wednesday[['Drug','Rank']].groupby('Drug').agg('sum')
rank_sum.columns=['Rank sum']

W = rank_sum.values.min()

z = (W - W_mean)/SE

# Get the P-value as above
DF=10000  # set a large degrees of freedom.
pvalue = 2* stats.t.cdf(z,DF) # get the P value from the CDF of a t-distribution
# Multiply by two, since this is two-tailed test


printmd('** Wednesday **')
printmd('$W_s$ {}'.format(W))
printmd('$W_s$ mean {}'.format(W_mean))
printmd('SE {}'.format(round(SE,2)))
printmd('$z$-score {}'.format(round(z,2)))
printmd('p-value {}'.format(round(pvalue,4)))


** Wednesday **

$W_s$ 59.0

$W_s$ mean 105.0

SE 13.23

$z$-score -3.48

p-value 0.0005

<a id='scipy'></a>
## Using scipy   

Let's run the same test using python's scipy

In [10]:
from scipy import stats

# Let's do the Wednesday sample as an example:   
# SELECT 'BDI Wednesday' FROM df WHERE 'Drug'='Alcohol'
x = df.loc[df['Drug']=='Alcohol']['BDI Wednesday'].values
# SELECT 'BDI Wednesday' FROM df WHERE 'Drug'='Ecstasy'
y = df.loc[df['Drug']=='Ecstasy']['BDI Wednesday'].values

results = stats.ranksums(x, y)
z = results[0]
pvalue = results[1]
printmd('** Wednesday **')
printmd('$z$-score {}'.format(round(z,2)))
printmd('p-value {}'.format(round(pvalue,4)))

# Let's do the Sunday sample :   
# SELECT 'BDI Sunday' FROM df WHERE 'Drug'='Alcohol'
x = df.loc[df['Drug']=='Alcohol']['BDI Sunday'].values
# SELECT 'BDI Sunday' FROM df WHERE 'Drug'='Ecstasy'
y = df.loc[df['Drug']=='Ecstasy']['BDI Sunday'].values

results = stats.ranksums(x, y)
z = results[0]
pvalue = results[1]
printmd('** Sunday **')
printmd('$z$-score {}'.format(round(z,2)))
printmd('p-value {}'.format(round(pvalue,4)))


** Wednesday **

$z$-score -3.48

p-value 0.0005

** Sunday **

$z$-score -1.1

p-value 0.273

<a id='MW'></a>
## Mann-Whitney test     

As mentioned before, the Mann-Whitney test is equivalent to the Wilcoxon rank-sum test (above). This is based on a test statistic $U$,

$ U = n_1 n_2 + \frac{\displaystyle n_1(n_1+1)}{\displaystyle 2} - R_1$,   

where $n_i$ is the sample size for the $i^\textrm{th}$ group and $R_1$ is the sum of ranks for group 1.   

In this case, the rank sums for Sunday are

In [11]:
rank_sum_sunday

,Rank sum
Drug,
Alcohol,90.5
Ecstasy,119.5


In [12]:
R1=rank_sum_sunday['Rank sum'][1]
printmd('**Sunday**')
printmd('Rank sum for group 1 (Ecstasy data) is {}'.format(R1))

n1=10
n2=10
U = n1*n2 + n1*(n1+1)/2 - R1
printmd('Test statistic, U = {}'.format(U))

**Sunday**

Rank sum for group 1 (Ecstasy data) is 119.5

Test statistic, U = 35.5

<a id='MW_scipy'></a>
## Mann-Whitney test with scipy   
Let's perform the test with Python's scipy   

In [13]:
# Let's do the Sunday sample :   
# SELECT 'BDI Sunday' FROM df WHERE 'Drug'='Alcohol'
x = df.loc[df['Drug']=='Alcohol']['BDI Sunday'].values
# SELECT 'BDI Sunday' FROM df WHERE 'Drug'='Ecstasy'
y = df.loc[df['Drug']=='Ecstasy']['BDI Sunday'].values

stats.mannwhitneyu(x, y, use_continuity=True, alternative=None)

MannwhitneyuResult(statistic=35.5, pvalue=0.14304106361646457)

The pvalue for Sunday is 0.14 (higher than a significance of 0.05) in agreement with the Wilcoxon rank-sum test, above.